In [53]:
import btrack
import napari
from skimage.io import imread
import csv
import numpy as np

load your segmentation data

In [66]:
segmentation = imread('/napari/358_61pos.tif')
segmentation = segmentation[0:80]

create btrack objects (with properties) from the segmentation data. (you can also calculate properties, based on scikit-image regionprops)

In [67]:
objects = btrack.utils.segmentation_to_objects(segmentation, properties=('area', ))

[INFO][2024/04/11 04:44:30 PM] Localizing objects from segmentation...
progress: 100%|█████████████████████████████████| 80/80 [00:10<00:00,  7.91it/s]
[INFO][2024/04/11 04:44:40 PM] Objects are of type: <class 'dict'>
[INFO][2024/04/11 04:44:40 PM] ...Found 18322 objects in 80 frames.


initialise a tracker session using a context manager

In [68]:
with btrack.BayesianTracker() as tracker:
    tracker.configure('/R/napari/config.json')
    tracker.append(objects)
    tracker.track_interactive(step_size=100)
    tracker.optimize()
    tracker.export('/R/napari/temp/tracks.h5', obj_type='obj_type_1')
    tracks = tracker.tracks
    #data, properties, graph = tracker.to_napari()

[INFO][2024/04/11 04:44:41 PM] Loaded btrack: /opt/anaconda3/envs/python/lib/python3.12/site-packages/btrack/libs/libtracker.dylib
[INFO][2024/04/11 04:44:41 PM] Starting BayesianTracker session
[INFO][2024/04/11 04:44:41 PM] Loading configuration file: /Users/farhad/Desktop/R/napari/config.json
[INFO][2024/04/11 04:44:41 PM] Objects are of type: <class 'list'>
[WARNING][2024/04/11 04:44:42 PM] `track_interactive` will be deprecated. Use `track` instead.
[INFO][2024/04/11 04:44:42 PM] Starting tracking... 
[INFO][2024/04/11 04:44:42 PM] Update using: ['MOTION']
[INFO][2024/04/11 04:44:42 PM] Tracking objects in frames 0 to 80 (of 80)...
[INFO][2024/04/11 04:44:43 PM]  - Timing (Bayesian updates: 17.96ms, Linking: 0.51ms)
[INFO][2024/04/11 04:44:43 PM]  - Probabilities (Link: 0.98954, Lost: 0.96091)
[INFO][2024/04/11 04:44:43 PM] SUCCESS.
[INFO][2024/04/11 04:44:43 PM]  - Found 416 tracks in 80 frames (in 0.0s)
[INFO][2024/04/11 04:44:43 PM]  - Inserted 0 dummy objects to fill tracking 

GLPK Integer Optimizer 5.0
1664 rows, 1306 columns, 1781 non-zeros
1306 integer variables, all of which are binary
Preprocessing...
832 rows, 1306 columns, 1781 non-zeros
1306 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 832
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
832 rows, 1306 columns, 1781 non-zeros
*     0: obj =   4.912923154e+02 inf =   0.000e+00 (100)
*    99: obj =   3.544186802e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+    99: mip =     not found yet >=              -inf        (1; 0)
+    99: >>>>>   3.544186802e+02 >=   3.544186802e+02   0.0% (1; 0)
+    99: mip =   3.544186802e+02 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND


In [62]:
#viewer = napari.Viewer()
#viewer.add_tracks(data, properties=properties, graph=graph)
#napari.run()

In [69]:
with open('/napari/115-tracking/tracks_358-61pos.csv', 'w', newline='') as file:
	writer = csv.writer(file)
	for x in tracks:
		writer.writerow([x, x.ID, x.t, x.x, x.y, x.root, x.parent, x.children, x.generation, x.fate])